# Game Score Notebook

## References and Links

### All of these metrics were created and scaled to approximate a corilation to Points in a game. 
Inspired by this article I found about a metric that was developed in 2016, Dom Luszczyszyn which is intended to provide a single number that approximates a player’s performance in a given game. In that original story, he included the formula for the metric and how he arrived at it, so it’s inspired various adaptations since then.
https://hockey-graphs.com/2016/07/13/measuring-single-game-productivity-an-introduction-to-game-score/

The stats I used are goals, primary assists, secondary assists, shots on goal, blocked shots, penalty differential, faceoffs, 5-on-5 corsi differential, 5-on-5 goal differential.

Player Game Score = (0.75 * G) + (0.7 * A1) + (0.55 * A2) + (0.075 * SOG) + (0.05 * BLK) + (0.15 * PD) – (0.15 * PT) + (0.01 * FOW) – (0.01 * FOL) + (0.05 * CF) – (0.05 * CA) + (0.15 * GF) – (0.15* GA)



## Adjusted / Simplified
- found on the blog frshice.substack.com
- Bailey Johnson created her simplified version to work on college hockey data because not all of the factors (specifically the defensive portion of the Corsi Metric) is not track or is not available for NCAA games

### Original
For clarity’s sake, this was my original formula after removing stats I didn’t have access to from Dom’s: Player Game Score = (0.75*G)+(0.7*A1)+(0.55*A2)+(0.075*SOG)+(0.05*BLK)+(0.01*FOW)–(0.01*FOL)+(0.15*GF)–(0.15*GA)

### Bailey Final
- I also followed Shawn’s method from his NWHL game score work and used league-wide power-play percentage to weight the impact of taking a penalty.

- Dom scaled his formula down 75% to make the game scores roughly equivalent to points so people would be familiar with what the game score represented, and I kept to that methodology here because I used some of the same weights he did. Also keep in mind that the goals for and goals against are just goals scored at even strength — it does not include special teams, empty-net or extra-attacker goals.

Player Game Score = (G*0.75)+(A1*0.715)+(A2*0.555)+(SOG*0.075)+(BLK*0.05)+(FOW*0.01)-(FOL*0.01)+(GF*0.15)-(GA*0.15)-(PNT*0.138)

### PNT*.138
- FROM SOURCE: I took the frequency of powerplay goals to penalties, otherwise known as PP%. (https://hockey-graphs.com/2018/03/22/an-introduction-to-nwhl-game-score/)
- this is her attempt to account for penalties - CHN NCAA stats don't include penalties drawn 
- PNT is penalties taken so I will want to grab the penalty incidents, not the minutes. 
- The 0.138 factor come from the league's power play percentage 
    - Want to use a static figure like overall average PP % for entire NCAA 
        - could update it to be dynamic and create a new average every time the data is called

        - IDEA: Could create a custom factor for each team in each game 
            - teams power play effectiveness can vary greatly, as can a teams PK%
            - Take the each teams previous success on PP or even on both PP and PK
            - Compare to NCAA wide average
            - get a factor that could be used and could scale the danger of taking a penalty based on how good the opponent is on PP or how poor your team is on PP


In [ ]:
## Blocking out the Game Score formula

#### METRIC Formula
# 
# METRIC = SCORE [ (Goals*0.75) + (Assist1*0.715) + (Assist2*0.555) ] 
#               + SHOTS [ (Shot_On_Net*0.075) + (Shots_Off_Net*0.075) - (Shots_Blocked*0.075) ]
#               + FACEOFFS [ (Faceoff_Wins*0.01) - (Faceoff_Losses*0.01) ]
#               + TEAM [ (Goals_For_Team*0.15) + (Goals_Against_Team*0.15) ]
#               -  [ (Penalties_Taken * Overall_PP_Success_Rate) ]

# Map to each factor in the Game_Stats Database

## Calculating a Game Score for each player on each team for each game
Each Game has a unique Game_ID column in every relevant table

scoring_summary table
    Goals - in scoring_summary - count of player's name in Player Column
    Assist1 - scoring_summary - Count in Assist1
    Assist2 - scoring_summary - Count in Assist2


abdvanced_metrics_combined table
    Shots_On_Net -  - EVEN_Saved + EVEN_Goals
    Shots_Off_Net = EVENE_Miss
    Shots_Blocked = EVEN_Block

    Defensive_Blocks = D_Blocks

player_stats
    Faceoff_Wins = FO_W
    Faceoff_Loses = FO_L

### This takes into account team goals for and goals against but only counts Even strength goals in close games (+/- 1)
- can get it from the advanced metrics combined

advanced_metrics_
    Goals_For_Team = SUM of CLOSE_Goals grouped by Team -NOTE
#### NOTE- need to filter out any rows in the advanced_metrics_combined with player = 'TOTAL'
    Goals_Against_Team - do the same but for the opposing team - if Game_ID matches and Team =/= the player's team

penalty_summary
    Penalties_Taken = Count of Player's name in 



In [ ]:

import sqlite3
import pandas as pd
import numpy as np

import sqlite3

db_path ='../data/2023_YTD_Game_Stats.db'
conn = sqlite3.connect(db_path)


# Connect to the provided database

cursor = conn.cursor()

# Retrieve the list of tables in the database
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
tables = [table[0] for table in tables]

# Retrieve the columns from each table to get a better understanding of the data structure
table_columns = {}
for table in tables:
    columns = cursor.execute(f"PRAGMA table_info({table});").fetchall()
    table_columns[table] = [column[1] for column in columns]

# table_columns


# Create an empty dataframe to store the data
compiled_df = pd.DataFrame()

## Get Shot Data

In [ ]:
# Update the code to clean the 'advanced_metrics_combined' table by removing 'TOTAL' rows

# Load the 'advanced_metrics_combined' table into a DataFrame excluding 'TOTAL' rows
df_amc = pd.read_sql("SELECT * FROM advanced_metrics_combined WHERE player != 'TOTAL'", conn)

# Convert columns to numeric for calculations (replacing empty strings with 0)
columns_to_convert = ['EVEN_Saved', 'EVEN_Goals', 'EVEN_Miss', 'EVEN_Block', 'D_Blocks']
for col in columns_to_convert:
    df_amc[col] = pd.to_numeric(df_amc[col].replace('', '0'))

# Calculate required columns
df_amc['Shots_On_Net'] = df_amc['EVEN_Saved'] + df_amc['EVEN_Goals']
df_amc['Shots_Off_Net'] = df_amc['EVEN_Miss']
df_amc['Shots_Blocked'] = df_amc['EVEN_Block']
df_amc['Defensive_Blocks'] = df_amc['D_Blocks']

# Create a DataFrame with only the desired columns
# shots_df = df_amc[['Player', 'Team', 'Shots_On_Net', 'Shots_Off_Net', 'Shots_Blocked', 'Defensive_Blocks', 'Game_ID']]

# Create a DataFrame with only the desired columns - LEAVE OFF TEAM AND TRY TO ADD IT LATER
shots_df = df_amc[['Player', 'Team', 'Shots_On_Net', 'Shots_Off_Net', 'Shots_Blocked', 'Defensive_Blocks', 'Game_ID']]

# Display the column names of the shots_df
shots_df.columns

shots_df.head(25)

# Merge into the compiled_df
compiled_df = shots_df.copy()

# # Team Name Split
# team_name_split = compiled_df['Game_ID'].str.split('-').str[-2:]
# compiled_df['Temp_Team_1'] = team_name_split.str[0]
# compiled_df['Temp_Team_2'] = team_name_split.str[1]

# # If Home/Away is Away Then Team is Temp_Team_1
# # If Home/Away is Home Then Team is Temp_Team_2
# compiled_df['Team'] = np.where(compiled_df['Home/Away'] == 'Away', compiled_df['Temp_Team_1'], compiled_df['Temp_Team_2'])

# # Drop Temp Columns
# compiled_df.drop(['Temp_Team_1', 'Temp_Team_2'], axis=1, inplace=True)


compiled_df.head(45)


# Use Master_Roster To Get Player to Team Mapping

In [ ]:
# ## See column names from master_roster table
# cursor.execute("SELECT * FROM master_roster")
# col_names = [description[0] for description in cursor.description]
# col_names

In [ ]:
# # Extracting all the player names from the master_roster table
# all_player_names_master_roster = set(pd.read_sql("SELECT Clean_Name FROM master_roster", conn)['Clean_Name'])

# # Finding the intersection of player names between compiled_df and master_roster
# common_names = set(compiled_df['Player']).intersection(all_player_names_master_roster)

# # Take out those annoying character if there are any
# # Replace non-breaking spaces with regular spaces in the player names
# compiled_df['Player'] = compiled_df['Player'].str.replace(u'\xa0', u' ')


# # Finding names in compiled_df that are not in master_roster
# missing_names = set(compiled_df['Player']) - all_player_names_master_roster

# len(common_names), len(missing_names), list(missing_names)[:10]  # Displaying the first 10 missing names


In [ ]:
# # Create a DataFrame with the missing names
# missing_names_df = compiled_df[compiled_df['Player'].isin(missing_names)]

# missing_names_df.head(25)

# ## unique Game_IDs in the missing_names_df
# missing_names_df['Game_ID'].nunique()

In [ ]:
## How many Nan Values are there in the Team Column?
compiled_df['Team'].isna().sum()

# Value Count of the Team Column
# compiled_df['Team'].value_counts()

# Get Goal, Assist1 and Assist2
- Working

In [ ]:
# Separately calculate Goals, Assist1, and Assist2

# Goals
goals_df = pd.read_sql("""
SELECT 
    Player,
    Game_ID,
    COUNT(Player) AS Goals
FROM 
    scoring_summary
GROUP BY 
    Player, Game_ID;
""", conn)

# Assist1
assist1_df = pd.read_sql("""
SELECT 
    Assist1 AS Player,
    Game_ID,
    COUNT(Assist1) AS Assist1
FROM 
    scoring_summary
WHERE 
    Assist1 IS NOT NULL AND Assist1 != ''
GROUP BY 
    Assist1, Game_ID;
""", conn)

# Assist2
assist2_df = pd.read_sql("""
SELECT 
    Assist2 AS Player,
    Game_ID,
    COUNT(Assist2) AS Assist2
FROM 
    scoring_summary
WHERE 
    Assist2 IS NOT NULL AND Assist2 != ''
GROUP BY 
    Assist2, Game_ID;
""", conn)


# goals_df.head(25)
# Merge the dataframes together
merged_df = goals_df.merge(assist1_df, on=['Player', 'Game_ID'], how='outer')
merged_df = merged_df.merge(assist2_df, on=['Player', 'Game_ID'], how='outer')

# Fill NaN values with 0
merged_df = merged_df.fillna(0)

# Convert columns to int
merged_df['Goals'] = merged_df['Goals'].astype(int)
merged_df['Assist1'] = merged_df['Assist1'].astype(int)
merged_df['Assist2'] = merged_df['Assist2'].astype(int)

# # Sort by Player
goal_assist_df = merged_df.sort_values(by=['Player'])

# # Display the first few rows
goal_assist_df.head(20)



# # Merge the Goal and Assist info into the compiled_df join left
# compiled_df = pd.merge(compiled_df, goal_assist_df, on=['Player', 'Game_ID'], how='outer')


compiled_df.sample(10)
compiled_df.info()



# # Histogram of the compiled_df
# compiled_df.hist()

In [ ]:
## Value count of Teams
compiled_df['Team'].value_counts()

# How many Nan Values are there in the Team Column?
compiled_df['Team'].isna().sum()

In [ ]:
# Replace non-breaking spaces with regular spaces in the Player column of compiled_df
compiled_df['Player'] = compiled_df['Player'].str.replace('\xa0', ' ')

# Merge the dataframes again
new_merged_df = pd.merge(compiled_df, goal_assist_df, on=['Player', 'Game_ID'], how='left')

# Fill NaN values with 0 for the Goals, Assist1, and Assist2 columns
new_merged_df[['Goals', 'Assist1', 'Assist2']] = new_merged_df[['Goals', 'Assist1', 'Assist2']].fillna(0).astype(int)

# Display the first few rows of the merged dataframe
merged_df.head()

new_merged_df.info()

# RENAME BACK TO COMPILED_DF FOR USE LATER
compiled_df = new_merged_df.copy()

In [ ]:
# new_merged_df['Goals'].hist()

# new_merged_df.sample(20)

# Value Counts of Teams
compiled_df['Team'].value_counts()

# How many Nan values are in the Team column?
compiled_df['Team'].isna().sum()

# 

In [ ]:
### OUTPUT CSVS

# # Save the compiled_df to a csv
compiled_df.to_csv('../TEMP/PRELIM_compiled_df.csv', index=False)

# # Save the goal_assist_df to a csv
goal_assist_df.to_csv('../TEMP/PRELIM_goal_assist_df.csv', index=False)

In [ ]:
# Calculate Faceoff Metrics for each player in each game

query_faceoff_metrics = """
SELECT 
    Player,
    Game_ID,
    FOW AS Faceoff_Wins,
    FOL AS Faceoff_Losses
FROM 
    player_stats;
"""

# Create a DataFrame with the results



faceoff_metrics_data = cursor.execute(query_faceoff_metrics).fetchall()

# Preview the first few rows of the result
faceoff_metrics_data[:5]

# Create DataFrame with the results
faceoff_metrics_df = pd.DataFrame(faceoff_metrics_data, columns=['Player', 'Game_ID', 'Faceoff_Wins', 'Faceoff_Losses'])

# Take out those annoying character if there are any
# Replace non-breaking spaces with regular spaces in the Player column of compiled_df
faceoff_metrics_df['Player'] = faceoff_metrics_df['Player'].str.replace('\xa0', ' ')

# Replace non-breaking spaces with regular spaces in the Player column of compiled_df
compiled_df['Team'] = compiled_df['Team'].str.replace('\xa0', ' ')

# Drop old indicator column to make way for new one
# compiled_df = compiled_df.drop(columns=['_merge'])
compiled_df = compiled_df.merge(faceoff_metrics_df, on=['Player', 'Game_ID'], how='inner', indicator=True)

# Histogram of the compiled_df
compiled_df.hist()

compiled_df.info()
# compiled_df.head(25)

# compiled_df.columns

# compiled_df['Team'].value_counts()

# compiled_df['Team'].isna().sum()

### Overall PP Success

In [ ]:
### Find the Overall Power Play Success rate for the entire database

# Count the total number of Power Play (PP) goals from the scoring_summary table.
pp_goals_count = cursor.execute("SELECT COUNT(*) FROM scoring_summary WHERE PP != '';").fetchone()[0]

# Count the total number of Power Plays from the penalty_summary table.
total_pp_count = cursor.execute("SELECT COUNT(*) FROM penalty_summary;").fetchone()[0]

# Calculate the Power Play success rate.
pp_success_rate = pp_goals_count / total_pp_count


## OVERALL NCAA WIDE POWER PLAY SUCCESS RATE TO USE IN FACTOR
pp_success_rate


In [ ]:
# even_strength_goals_refined_df.shape

# Drop indicator column before new merge
compiled_df = compiled_df.drop(columns=['_merge'])





# # Save the compiled_df to a csv
compiled_df.to_csv('../TEMP/NEW_PRELIM_compiled_df.csv', index=False)

compiled_df['Team'].value_counts()

compiled_df['Team'].isna().sum()

# compiled_df.sample(20)

# Team Goals Component

In [ ]:
## Get the Team_Goals_For and Team_Goals_Against for each player in each game
## Only include EVEN_STRENGTH goals
## Access the EVEN_STRENGTH goals advanced_metrics_combined table from the TOTAL row from each team and apply it to each player in the game
## EVEN Strength goals for the team should be stored in 'Goals_For_Team' and the opponent's EVEN Strength goals should be stored in 'Goals_Against_Team'

# Extracting EVEN_STRENGTH goals (where PP column is blank) for each team
even_strength_goals_query_refined = """
SELECT 
    Game_ID,
    Team,
    COUNT(*) AS Goals_For_Team
FROM 
    scoring_summary
WHERE 
    PP = ''
GROUP BY 
    Game_ID, Team
"""

even_strength_goals_refined_df = pd.read_sql(even_strength_goals_query_refined, conn)

# Correct the values in the Team column so we can effectively join the dataframes

# Create a new column for the row number grouped by Game_ID
even_strength_goals_refined_df['row_number'] = even_strength_goals_refined_df.groupby('Game_ID').cumcount() + 1

# Function to assign the correct team name based on the row number
def assign_team_name(row):
    return row['Team_1'] if row['row_number'] == 1 else row['Team_2']

# Splitting the Game_ID column again to extract team names
team_names_split = even_strength_goals_refined_df['Game_ID'].str.split('-').str[-2:]
even_strength_goals_refined_df['Team_1'] = team_names_split.str[0]
even_strength_goals_refined_df['Team_2'] = team_names_split.str[1]

# Applying the function to assign the correct team name
even_strength_goals_refined_df['Team'] = even_strength_goals_refined_df.apply(assign_team_name, axis=1)

# Dropping the extra columns again
even_strength_goals_refined_df.drop(columns=['Team_1', 'Team_2', 'row_number'], inplace=True)

# Creating a new column 'Opponent_Goals' which will be used to determine the goals against each team
even_strength_goals_refined_df = even_strength_goals_refined_df.merge(
    even_strength_goals_refined_df[['Game_ID', 'Team', 'Goals_For_Team']], 
    on='Game_ID',
    suffixes=('', '_Opponent')
)

# Filtering out rows where Team is the same as the opponent team
even_strength_goals_refined_df = even_strength_goals_refined_df[even_strength_goals_refined_df['Team'] != even_strength_goals_refined_df['Team_Opponent']]

# Assigning the 'Goals_For_Team_Opponent' to a new column 'Goals_Against_Team'
even_strength_goals_refined_df['Goals_Against_Team'] = even_strength_goals_refined_df['Goals_For_Team_Opponent']

# Dropping unnecessary columns
even_strength_goals_refined_df.drop(columns=['Team_Opponent', 'Goals_For_Team_Opponent'], inplace=True)

even_strength_goals_refined_df.head(25)
# Take out those annoying character if there are any
# Replace non-breaking spaces with regular spaces in the Player column of compiled_df
even_strength_goals_refined_df['Team'] = even_strength_goals_refined_df['Team'].str.replace('\xa0', ' ')

## SAVE A CSV 
even_strength_goals_refined_df.to_csv('../TEMP/NEW_PRELIM_even_strength_goals_refined_df.csv', index=False)

# # Displaying the dataframe with the newly computed 'Goals_Against_Team' values
even_strength_goals_refined_df.sample(25)


# # print(compiled_df.shape)


In [ ]:
# Merge the dataframes on Game_ID and Team
merged_df_new = pd.merge(compiled_df, even_strength_goals_refined_df, on=['Game_ID', 'Team'], how='outer')

# Display the first few rows of the merged dataframe
merged_df_new.head(25)

merged_df_new.info()


In [ ]:

# # Convert columns to int
# int_cols = [ 'Shots_On_Net', 'Shots_Off_Net', 'Shots_Blocked',
#        'Defensive_Blocks',  'Goals', 'Assist1', 'Assist2',
#        'Faceoff_Wins', 'Faceoff_Losses', 'Goals_For_Team',
#        'Goals_Against_Team']

# for col in int_cols:
#     compiled_df[col] = compiled_df[col].astype(int)



# print(f'New Shape: {compiled_df.shape}')

# ####
# # # Merge into the compiled_df
# compiled_df = compiled_df.merge(even_strength_goals_refined_df, on=['Game_ID', 'Team'], how='outer', indicator=True)

# compiled_df.columns

# # compiled_df['Goals'].hist()
# # compiled_df['Assist1'].hist()
# compiled_df['Assist2'].hist()

# compiled_df.sample(25)

# # Team value count
# compiled_df['Team'].value_counts()



In [ ]:
# SHow just the rows where the Goals_Against_Team is NaN
trouble_df = merged_df_new[merged_df_new['Goals_Against_Team'].isna()]

# Value Counts of Teams
trouble_df['Team'].value_counts()

# I think what is going on is thes are instanced when the player was not on the roster the day of the game?

## Do any of these have values in Goals or Assists?
trouble_df[trouble_df['Goals'] > 0]

trouble_df[trouble_df['Assist1'] > 0]

## Could be games when no even strength goals were scored - That must be it. fill with 0
merged_df_new['Goals_Against_Team'] = merged_df_new['Goals_Against_Team'].fillna(0)
# Same for Goals_For_Team
merged_df_new['Goals_For_Team'] = merged_df_new['Goals_For_Team'].fillna(0)

In [ ]:
merged_df_new.hist()

merged_df_new.info()

In [ ]:
# # Joining the dataframe with itself to compute EVEN_STRENGTH goals against each team
# team_goals_refined_df = even_strength_goals_refined_df.merge(
#     even_strength_goals_refined_df, on='Game_ID', suffixes=('', '_Opponent')
# )

# # Filtering out rows where Team is the same as the opponent team
# team_goals_refined_df = team_goals_refined_df[team_goals_refined_df['Team'] != team_goals_refined_df['Team_Opponent']]

# # Dropping unnecessary columns and renaming for clarity
# team_goals_refined_df = team_goals_refined_df[['Game_ID', 'Team', 'Goals_For_Team', 'Goals_For_Team_Opponent']]
# team_goals_refined_df = team_goals_refined_df.rename(columns={'Goals_For_Team_Opponent': 'Goals_Against_Team'})

# # Joining with advanced_metrics_combined to assign the values to each player
# player_goals_refined_df = player_goals_df.merge(team_goals_refined_df, on=['Game_ID', 'Team'], how='left')

# # Display the first few rows of the resulting dataframe
# player_goals_refined_df[['Player', 'Game_ID', 'Team', 'Goals_For_Team', 'Goals_Against_Team']].head(25)


# Penalty Component

In [ ]:
## Penalty Component

# Count the number of penalties taken by each player in each game
penalty_count_query = """
SELECT
    Game_ID,
    Player,
    COUNT(*) AS Penalties_Taken
FROM
    penalty_summary
WHERE
    Player != 'TOTAL'
GROUP BY
    Game_ID, Player;
"""

## Rename the new version of the df back to this name to use later

compiled_df = merged_df_new.copy()


# Create a DataFrame with the results
penalty_count_df = pd.read_sql(penalty_count_query, conn)

# Preview the first few rows of the result
penalty_count_df.head()

penalty_count_df['Penalties_Taken'].hist()

# Convert to int
penalty_count_df['Penalties_Taken'] = penalty_count_df['Penalties_Taken'].astype(int)

# penalty_count_df.describe()

# merge it into the compiled_df
compiled_df = compiled_df.merge(penalty_count_df, on=['Player', 'Game_ID'], how='left')

# Fill NaN values in 'Penalties_Taken' with 0
compiled_df['Penalties_Taken'].fillna(0, inplace=True)

# compiled_df.head(25)


# compiled_df.hist()

compiled_df.columns

# Output CSV
compiled_df.to_csv('../TEMP/GAME_SCORE_compiled_df.csv', index=False)


compiled_df.info()

In [ ]:
# Fill NaN values in 'Penalties_Taken' with 0
compiled_df['Penalties_Taken'].fillna(0, inplace=True)

# Calculate each component of the game score formula
player_goals_refined_df = compiled_df.copy()

# SCORE Component
player_goals_refined_df['SCORE'] = (player_goals_refined_df['Goals'] * 0.75) + \
                                  (player_goals_refined_df['Assist1'] * 0.715) + \
                                  (player_goals_refined_df['Assist2'] * 0.555)

# SHOTS Component
player_goals_refined_df['SHOTS'] = (player_goals_refined_df['Shots_On_Net'] * 0.075) + \
                                  (player_goals_refined_df['Shots_Off_Net'] * 0.075) - \
                                  (player_goals_refined_df['Shots_Blocked'] * 0.075)

# FACEOFFS Component
player_goals_refined_df['FACEOFFS'] = (player_goals_refined_df['Faceoff_Wins'] * 0.01) - \
                                     (player_goals_refined_df['Faceoff_Losses'] * 0.01)
# Fill NaN values in 'Faceoff_Wins' and 'Faceoff_Losses' with 0
player_goals_refined_df['FACEOFFS'].fillna(0, inplace=True)


# TEAM Component
player_goals_refined_df['TEAM'] = (player_goals_refined_df['Goals_For_Team'] * 0.15) - \
                                 (player_goals_refined_df['Goals_Against_Team'] * 0.15)
# Fill


# PENALTIES Component -  Negative because it is a penalty
player_goals_refined_df['PENALTIES'] = (player_goals_refined_df['Penalties_Taken'] * pp_success_rate) * -1

## Fill all Nan Values with 0
player_goals_refined_df.fillna(0, inplace=True)

# Calculate the overall game score
player_goals_refined_df['Game_Score'] = player_goals_refined_df['SCORE'] + \
                                       player_goals_refined_df['SHOTS'] + \
                                       player_goals_refined_df['FACEOFFS'] + \
                                       player_goals_refined_df['TEAM'] - \
                                       player_goals_refined_df['PENALTIES']

# Display the first few rows of the dataframe with the newly computed 'Game_Score' values
player_goals_refined_df[['Player', 'Game_ID', 'Team', 'Game_Score']].head(25)



# Save to CSV
player_goals_refined_df.to_csv('../TEMP/GAME_SCORE_player_goals_refined_df.csv', index=False)


# Pivot the big Game Score Table into individual teams

In [ ]:
# Generate team-specific tables using a pivot table
team_tables = {}

unique_teams = player_goals_refined_df['Team'].unique()

for team in unique_teams:
    team_df = player_goals_refined_df[player_goals_refined_df['Team'] == team]
    pivot_table = team_df.pivot(index='Player', columns='Game_ID', values='Game_Score')
    team_tables[team] = pivot_table


# Print List of Teams
unique_teams


# Fill NaN values with 0
for team in unique_teams:
    team_tables[team] = team_tables[team].fillna(0)

# Display the first team's table as an example
# team_tables[unique_teams[21]].head(40)

### MSU Is In Index Position 21 in this case outputing csv for MSU
# team_tables[unique_teams[21]] = team_tables[unique_teams[21]].fillna(0)
# team_tables[unique_teams[21]].to_csv('../data/Nov_2_MSU_GAME_SCORES.csv')







In [ ]:
## 

# Pivot Tables of Game Scores for Every Player and Every Team should all be stored now if it ran cleanly 

In [ ]:
## 

In [ ]:
unique_teams

In [ ]:
# Display the first team's table as an example
team_tables[unique_teams[7]].head(40)

ohio_state_df = team_tables[unique_teams[7]].copy()

# Clean Column Headers
# split on - and take the last 2 elements
# ohio_state_df.columns = ohio_state_df.columns.str.split('-').str[-2:]

ohio_state_df.head(25)

# Get a list of the column names
column_names = ohio_state_df.columns.tolist()
# Clean up column names by Removing 'Ohio State' and any digits of dashes
column_names = [name.replace('Ohio State', '') for name in column_names]
column_names = [name.replace('-', '') for name in column_names]
column_names = [name.replace(' ', '') for name in column_names]

# Use regex to remove any digits
import re

column_names = [re.sub(r'\d+', '', name) for name in column_names]



# Replace the column names
ohio_state_df.columns = column_names

ohio_state_df.head(25)

# Save to CSV
ohio_state_df.to_csv('../TEMP/GAME_SCORE_OHIO_STATE.csv', index=True)

### MSU Is In Index Position 21 in this case outputing csv for MSU
# team_tables[unique_teams[21]] = team_tables[unique_teams[21]].fillna(0)
# team_tables[unique_teams[21]].to_csv('../data/Nov_2_MSU_GAME_SCORES.csv')



# One of THe Michigan Games is Absent From The OSU DATA

In [ ]:
ohio_state_df.head()

In [ ]:
# Create a list of dataframes from the team_tables dictionary
team_tables_list = [team_tables[team] for team in unique_teams]


## Save To an Excell File

In [ ]:
# Assuming unique_teams and team_tables are lists and of the same length
import xlsxwriter

# Create a workbook and add a worksheet for each team
workbook = xlsxwriter.Workbook('../data/Nov_2_GAME_SCORES.xlsx')

for team, table in team_tables_dict.items():
    # Create a new worksheet with the team name
    worksheet = workbook.add_worksheet(team)
    
    # Write the headers
    for col_num, column in enumerate(table.columns):
        worksheet.write(0, col_num, column)
    
    # Write the data
    for row_num, row_data in enumerate(table.values):
        for col_num, cell_data in enumerate(row_data):
            worksheet.write(row_num + 1, col_num, cell_data)  # +1 to skip the header

workbook.close()

# NOTE: The above code is structured to work with the provided mock data. Adjustments might be needed based on actual data.


In [ ]:
# Save the team tables to a dictionary
import pickle


In [ ]:
## Blocking out the Game Score formula

#### METRIC Formula
# 
# METRIC = SCORE [ (Goals*0.75) + (Assist1*0.715) + (Assist2*0.555) ] 
#               + SHOTS [ (Shot_On_Net*0.075) + (Shots_Off_Net*0.075) - (Shots_Blocked*0.075) ]
#               + FACEOFFS [ (Faceoff_Wins*0.01) - (Faceoff_Losses*0.01) ]
#               + TEAM [ (Goals_For_Team*0.15) + (Goals_Against_Team*0.15) ]
#               -  [ (Penalties_Taken * Overall_PP_Success_Rate) ]

## Where to find these values in my DB
scoring_summary
    - Goals
    -First Assist
    -Second Assist

player_stats
    - Shots on Goal
    - FOW
    - FOL
    - PIM 
        - (Maybe worth weighting differently based on period and time it was taken)
        - Penalty late in a close game hurts a team more than something taken in the first perios
        - a penalty that is taken when already short handed hurts much more than one 5-on-5
            - I should be able to seperate out these types of occurences in the data from penalty_summary

CAN'T Get Penalties Drawn from current data

### Formulating the final 3 factors
GA & GF should only use even strength goals - need to figure out how to filter those

Shots blocked (overall - defensive) can be found in advanced metrics as well as SOG, Offensive shots blocked and shots missed net for each of these situations (total, close, even and PP)





## Corsi Differnal = Corsi For - Corsi Defence
Corsi is an advanced statistic used in the game of ice hockey to measure shot attempt differential while at even strength play. This includes shots on goal, missed shots on goal, and blocked shot attempts towards the opposition's net minus the same shot attempts directed at your own team's net.

History
The Corsi number was named by Tim Barnes, a financial analyst from Chicago working under the pseudonym Vic Ferrari. He had heard former Buffalo Sabres general manager Darcy Regier talking about shot differential on the radio, and then proceeded to develop a formula to accurately display shot differential. Ferrari originally wanted to name it the Regier number, but he didn't think it sounded right. He then considered calling it the Ruff number after former Buffalo Sabres head coach Lindy Ruff but he didn't think that was appropriate either. Ferrari ended up searching Buffalo Sabres staff, found a picture of Jim Corsi, and chose his name because he liked Corsi's mustache.[1]

Formulae
Corsi For (CF) = Shot attempts for at even strength: Shots + Blocks + Misses[2]
Corsi Against (CA) = Shot attempts against at even strength: Shots + Blocks + Misses
Corsi (C) = CF - CA
Corsi For % (CF%) = CF / (CF + CA)
Corsi For % Relative (CF% Rel) = CF% - CFOff%
Corsi Per 60 Minutes at Even Strength (C/60) = (CF - CA) * 60 / TOI
Relative Corsi per 60 Minutes at Even Strength (Crel/60) = CF/60 - CFoff/60 = On-Ice Corsi For / 60 Minutes - Off-Ice Corsi For / 60 Minutes